# Robotics project 

In this project, 5 components were required :
- Vision 
- Global Navigation 
- Filtering 
- Motion Control 
- Local Navigation 

In this notebook, we'll briefly present how we incorporated them in the project. 

In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt

LENGTH = 32
WIDTH = 29
FREE = 0
OCCUPIED = 1

## Members of the team 
- Olivier Charrez  (271550)
- Arthur Chevalley (283178)
- Paco Mermoud     (290469)
- Tifanny Portela  (289149)

# Vision


At the beginning of the programm, we use a vision algorithm to determine the initial pose of the robot, the position of the goal and the obstacles.

We used 4 different masks to detect these objects:
- A red one to detect the obstacles 
- A blue one to detect the edges of the map and the goal
- A yellow and a green one to determine the pose of the robot 

For example, this algorithm yields to the following result:




When the code is running, we also use the vision as the measurement in the update step of our Kalman filter. 

# Global Navigation 

In this part, we implemented the A* algorithm to allow Thymio to avoid the obstacles without using the sensors to detect them.
In order to do that we discretized the world into a grid map with squared cells of size 2.5 cm. This map has a width of 80 cm (32 cells) and a length of 72.5 cm (32 cells).

To ease the implementation of the A* algorithm, we first increase the size of the obstacles by the radius of the robot, so that we can consider the robot as a point in the path planning algorithm.





## A* algorithm 
The A* algorithm expandes the node n with the smallest $f(n) = g(n) + h(n)$, where $g(n)$ denotes the cheapest path cost from the starting node and $h(n)$ the heuristic function.

We decided tu use the A* algorithm to find the optimal path to the goal, because if $h(n)$ is a consistent heuristic, then A* is optimally efficient among all optimal search algorithms using $h(n)$. Indeed, no other optimal algorithm using $h(n)$ is guaranteed to expand fewer nodes than A*. $h(n)$ is consistent if the $f(n)$ costs are non-decreasing : $f(n_j)\geq f(n_i) $ (going from a node i to a node j). 

The heuristic function has been chosen to be the estimation of the cost to reach the goal from node n (in a straight line without considering the obstacles). This specific heuristic function is consistant, because $h(n_i)\geq cost(n_i, n_j)+h(n_j)$, hence A* is optimally efficient. 

The following figure shows an example of the implementation of such an algorithm in a map with increased obstacles where:

- The white cells represent free cells
- The red cells represent occupied cells (obstacles)
- The blue nodes are the nodes corresponding to the optimal path
- The green circle represent the starting node
- The purple circle represents the goal node


In [ ]:

    

def increased_obstacles_map(occupancy_grid):
    
    nb_rows = len(occupancy_grid)
    nb_cols = len(occupancy_grid[0])
    increased_occupancy_grid = np.zeros([nb_rows+6, nb_cols+6])
    
    for i in range(nb_rows):
        for j in range(nb_cols):
            
            if occupancy_grid[i,j] == OCCUPIED:
                increased_occupancy_grid[i:i+7,j:j+7] = np.ones([7,7])
                
    
    final_occupancy_grid = increased_occupancy_grid[3:(LENGTH + 3),3:(WIDTH + 3)]
    return final_occupancy_grid

# Filtering : EKF 

The objective of this part is to estimate the pose of the robot from noisy sensors measurements (odometry and vision). In order to do this, we decided to use the extended kalman filter (EKF).

## Control model

Let's start by defining the discrete-time state-space model $x_{t+1} = f(x_t, u_t)$ where the state $x_t$ is a vector representing the pose of the robot (2D position and orientation), i.e. $x_t =  \begin{bmatrix} x_t\\ y_t\\\theta_t  \end{bmatrix}$, and $u_t$ denotes the control input (here the reading from a proprioceptive sensor). Hence, $u_t$ can be written as $u_t =  \begin{bmatrix} \Delta_{s_r}\\ \Delta_{s_l}  \end{bmatrix}$, where $\Delta_{s_r}$ and $\Delta_{s_l}$ are determined using the encoder readings of the right and left wheel.

Furthermore, let's define some useful variables

\begin{itemize}
    \item $\Delta_{s_r}$ : travelled distance for the right wheel 
    \item $\Delta_{s_l}$ : travelled distance for the left wheel 
    \item $\Delta s = \frac{\Delta_{s_r} + \Delta_{s_l}}{2}$
    \item $(\Delta x, \Delta y, \Delta \theta)$ : path travelled in the last sampling interval of fixed length $T_s$
    \item $\Delta \theta = \frac{\Delta_{s_r} - \Delta_{s_l}}{b}$ 
    \item $b$ : the distance between the two wheels of the differential drive robot 
\end{itemize}
$x_{t+1}= f(x_t, u_t) = f(x_t, y_t, \theta_t, \Delta_{s_r}, \Delta_{s_l}) =  \begin{bmatrix} x_{t+1}\\ y_{t+1}\\\theta_{t+1}  \end{bmatrix} = \begin{bmatrix} x_t + \Delta_s cos(\theta_t + \frac{\Delta \theta}{2})\\ y_t + \Delta_s sin(\theta_t + \frac{\Delta \theta}{2}) \\ \theta_t + \Delta \theta  \end{bmatrix}$
 

Since $f(x_t, u_t)$ is a non-linear function, we linearize it around the point ($\mu_{x_t}$,$\mu_{u_t}$). In order to do so, we need to calculate the following two jacobian matrices:

$F_x = \left.\frac{\partial f(x_t, u_t)}{\partial x_t} \right|_{(\mu_{x_t},\mu_{u_t})} = \begin{bmatrix} 
1 & 0 & -\Delta_s sin(\theta_t + \frac{\Delta \theta}{2}) \\ 
0 & 1 & \Delta_s cos(\theta_t + \frac{\Delta \theta}{2})\\ 
0 & 0 & 1
\end{bmatrix}$

$F_u = \left.\frac{\partial f(x_t, u_t)}{\partial u_t} \right|_{(\mu_{x_t},\mu_{u_t})} = \begin{bmatrix} 
\frac{1}{2} cos(\theta_t + \frac{\Delta \theta}{2}) - \frac{\Delta_s}{2b} sin(\theta_t + \frac{\Delta \theta}{2}) & \frac{1}{2} cos(\theta_t + \frac{\Delta \theta}{2}) + \frac{\Delta_s}{2b} sin(\theta_t + \frac{\Delta \theta}{2}) \\ 
\frac{1}{2} sin(\theta_t + \frac{\Delta \theta}{2}) + \frac{\Delta_s}{2b} cos(\theta_t + \frac{\Delta \theta}{2}) & \frac{1}{2} sin(\theta_t + \frac{\Delta \theta}{2}) - \frac{\Delta_s}{2b} cos(\theta_t + \frac{\Delta \theta}{2}) \\
\frac{1}{b} & - \frac{1}{b}
\end{bmatrix}$



The control input has an associated covariance matrix $R= \begin{bmatrix} k_r  & 0\\ 0 & k_l  \end{bmatrix}$, where $k_r$ and $k_l$ are constants to be determined experimentally. 

In order to determine these constants, we measured the time Thymio took to travel a known distance, while recording the speed values in Thymio units. From these measurements we computed: $k_r = 1.34 \cdot 10^{-4} m$ and $k_l = 8.3466 \cdot 10^{-5} m$.


The prediction step is:
\begin{itemize}
    \item $\overline{\mu}_{t} = f(\mu_{x_{t-1}},\mu_{u_{t-1}})$
    \item $\overline{\Sigma}_{t} = F_x \Sigma_{t-1} F_x^T + F_u R F_u^T$
\end{itemize}

## Measurement model


We chose the measurement model to be the following :

$y_t = H_t x_t + \epsilon_t = \begin{bmatrix} 1 & 0 & 0\\ 0 & 1 & 0\\ 0 & 0 & 1 \end{bmatrix} \begin{bmatrix} x_t\\ y_t\\\theta_t  \end{bmatrix} + \epsilon_t$, where $\epsilon_t$ is the measurment noise with zero mean and covariance $Q = \begin{bmatrix} q_x & 0 & 0\\ 0 & q_y & 0 \\ 0 & 0 & q_{\theta}  \end{bmatrix} $, where $q_x$, $q_y$ and $q_t$ are constants to be determined experimentally.

In order to determine these constants, we measured several known poses of Thymio using the camera. From these measurements we computed: $q_x = 2.8948 \cdot 10^{-4} m$, $q_y = 8.2668 \cdot 10^{-4} m$ and $q_{\theta} = 0.0029 rad$.


$z_t = \begin{bmatrix} z_{x_t}\\ z_{y_t}\\ z_{\theta_t}  \end{bmatrix}$ is the reading from the camera (exteroceptive sensor), also called the observation.

The kalman gain is defined as $K_t = \overline{\Sigma}_{t}H_t^T(H_t \overline{\Sigma}_{t} H_t^T + Q)^{-1} = \overline{\Sigma}_{t}( \overline{\Sigma}_{t} + Q)^{-1}$. 

Hence, the update step is :
\begin{itemize}
    \item $\mu_{t} = \overline{\mu}_{t} + K_t(z_t - \overline{\mu}_{t})$
    \item $\Sigma_{t} = (I-K_tH_t) \overline{\Sigma}_{t} = (I-K_t) \overline{\Sigma}_{t}$
\end{itemize}



# Motion Control 

We have two different motion control algorithms. The first one is used in the main programm and the second one is used in the local Navigation programm. 




# Local Navigation 

While navigating, Thymio uses its proximity sensors to avoid physical obstacles that can be put in its path at any point in time. We chose these obstacles to be squares.

Once Thymio detects the local obstacle, he follows it using its proximity sensors until he crosses the global path. While he follows the If a global obstacle 